#Importing Colab and necessary packages

In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
!pip install torchtext -qq
!pip install transformers evaluate wandb datasets accelerate -U -qq
!pip install emoji cleantext -qq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 66.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 80.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 49.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.1/267.1 kB 31.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.8/433.8 kB 8.5 MB/s eta 0:00:00


In [5]:
import random
import numpy as np
import joblib
from collections import Counter
from pathlib import Path
import sys
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset
import base64
from transformers.modeling_outputs import SequenceClassifierOutput
from transformers import PreTrainedModel, PretrainedConfig
from transformers import TrainingArguments, Trainer
from functools import partial
import wandb
import matplotlib.pyplot as plt
from datasets import load_dataset, DatasetDict
from transformers import TrainingArguments, Trainer
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import AutoConfig
from transformers import pipeline
import re
from cleantext import clean
import emoji
from datasets import load_dataset

##Path and defining folders

In [ ]:
#google/flan-t5-base

In [6]:
basepath = ("/content/drive/MyDrive/Applied_NLP/")
sys.path.append("/content/drive/MyDrive/Applied_NLP/Custom_Files/")
sys.path

['/content',
 '/env/python',
 '/usr/lib/python310.zip',
 '/usr/lib/python3.10',
 '/usr/lib/python3.10/lib-dynload',
 '',
 '/usr/local/lib/python3.10/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.10/dist-packages/IPython/extensions',
 '/root/.ipython',
 '/tmp/tmplt26ipij',
 '/content/drive/MyDrive/Applied_NLP/Custom_Files/']

In [7]:
basefolder = Path(basepath)
datafolder = basefolder/"Class_7/Home_Work_5/Data/"
modelfolder = basefolder/"Class_8/Home_Work_6/Model_Checkpoint/"
customfolder = basefolder/"Custom_Files/"

In [6]:
from CustomPreprocessorSpacy import SpacyPreprocessor

In [8]:
datafolder

PosixPath('/content/drive/MyDrive/Applied_NLP/Class_7/Home_Work_5/Data')

#Importing Data

In [9]:
df_train = pd.read_csv(datafolder/"emoticons_train.csv")

In [10]:
df_test = pd.read_csv(datafolder/"emoticons_test.csv")

##Labels in the data

In [56]:
labels = [label for label in df_train.columns if label not in ["Tweet","ID"]]
id2labels = {idx:label for idx, label in enumerate(labels)}
labels2id = {v:k for k,v in id2labels.items()}
labels2id

{'anger': 0,
 'anticipation': 1,
 'disgust': 2,
 'fear': 3,
 'joy': 4,
 'love': 5,
 'optimism': 6,
 'pessimism': 7,
 'sadness': 8,
 'surprise': 9,
 'trust': 10}

#Dropping,Preprocessing and Splitting

In [11]:
df_train.drop("ID",axis=1,inplace=True)
df_test.drop("ID", axis=1, inplace=True)

In [ ]:
#df_train["Tweet"] = [emoji.replace_emoji(text, replace='') for text in df_train["Tweet"]]
#df_train["Tweet"] = [re.sub(r'@\w+|#\w+', '', text)for text in df_train["Tweet"]]
#df_test["Tweet"] = [emoji.replace_emoji(text, replace='') for text in df_test["Tweet"]]
#df_test["Tweet"] = [re.sub(r'@\w+|#\w+', '', text)for text in df_test["Tweet"]]

In [12]:
df_train

,Tweet,anger,anticipation,disgust,fear,joy,love,optimism,pessimism,sadness,surprise,trust
0,“Worry is a down payment on a problem you may ...,0,1,0,0,0,0,1,0,0,0,1
1,Whatever you decide to do make sure it makes y...,0,0,0,0,1,1,1,0,0,0,0
2,@Max_Kellerman it also helps that the majorit...,1,0,1,0,1,0,1,0,0,0,0
3,Accept the challenges so that you can literall...,0,0,0,0,1,0,1,0,0,0,0
4,My roommate: it's okay that we can't spell bec...,1,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
7719,@BadHombreNPS @SecretaryPerry If this didn't m...,1,0,1,0,0,0,0,0,0,0,0
7720,Excited to watch #stateoforigin tonight! Come ...,0,0,0,0,1,0,1,0,0,0,0
7721,"Blah blah blah Kyrie, IT, etc. @CJC9BOSS leavi...",1,0,1,0,0,0,0,0,1,0,0
7722,#ThingsIveLearned The wise #shepherd never tru...,0,0,0,0,0,0,0,0,0,0,0


##Spacy Preprocessing

In [ ]:
sp = SpacyPreprocessor(model='en_core_web_sm', batch_size=64, lemmatize=True, lower=True,
                                    remove_stop=True, remove_punct=True, remove_email=True,
                                    remove_url=True, remove_num=False, stemming=False,
                                    add_user_mention_prefix=False, remove_hashtag_prefix=True, basic_clean_only=False)

In [13]:
X_train_total = df_train["Tweet"].tolist()

In [14]:
X_test = df_test["Tweet"].tolist()

In [16]:
y_test = df_train

In [17]:
y_train_total = df_train.drop("Tweet",axis=1).to_numpy().astype(float)
y_test = df_test.drop("Tweet",axis=1).to_numpy()

In [18]:
y_train_total

array([[0., 1., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 1., ..., 0., 0., 0.],
       ...,
       [1., 0., 1., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [21]:
type(y_train)

numpy.ndarray

In [19]:
y_test = np.where(y_test == "NONE",0,y_test).astype(float)

##Train-Valid Split

In [20]:
X_train,X_valid,y_train,y_valid = train_test_split(X_train_total,y_train_total,test_size=0.3, random_state=42, shuffle=True)

##Spacy preprocessor

In [ ]:
X_train_preprocessed = sp.fit_transform(X_train)
X_valid_preprocessed = sp.transform(X_valid)
X_test_preprocessed = sp.transform(X_test)

/content/drive/MyDrive/Applied_NLP/Custom_Files/CustomPreprocessorSpacy.py:83: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


##Data Dictionaries

In [ ]:
X_train_preprocessed

['@kmunie7 @karanesch helluva lot animated actual game >.<',
 "despite sharing o'neill article sadden gazza case fury sentence remark",
 '@mtailor 20 well tailor tailor know #navy vs. #blue $ 500 4 week #innacurate suit',
 'earth projection good happy true soul poisonous fucking snake #patronus #adder #offende',
 'patti sad stamp run sofa plenty love affection usual',
 '@wwe live event brock vs orton 2 saturday wwe network need #wwe @randyorton @heymanhustle #revenge',
 '@riotzxvl papercut sting stub ur toe like 10 sec',
 '@smendler 2   watch ppl fill w/ impotent rage time lead',
 '@tbergg1 good rn go to horrid week',
 'watch amazing broadcast @kana_blender   #musically',
 '@twojacksdetail @bluelivesmtr rare officer shoot regard want conscience #incite',
 '@bennymcnugget sad upset idk',
 '@orbsofjoy plan date like date u find pleasing smth fuckign\\n\\n10/10 child grow',
 'ya girl',
 '@itejasmehta @qtalker pleasure 😊',
 'incomplete state tails possess immense physical strength remarkab

In [32]:
type(validset['labels'][0])

list

In [24]:
trainset = Dataset.from_dict({
    'texts': X_train,
    'labels': y_train
})

validset = Dataset.from_dict({
    'texts': X_valid,
    'labels': y_valid
})

testset = Dataset.from_dict({
    'texts': X_test,
    'labels': y_test
})

In [33]:
from joblib import dump, load
# Save datasetsdump(train_set, 'train_set.joblib')


In [34]:
basepath

'/content/drive/MyDrive/Applied_NLP/'

In [ ]:
basepath+

In [61]:
tokenized_dataset_test_gflan[0]

{'labels': tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 'input_ids': tensor([ 3320,   188,    26,    29,   152,   834,   834,   940,  3840,   834,
           834,  3320,   188,     7,  3774, 10358,   518,  1273,  1008,    17,
          3516,  2557,  9102,    19,    30,   165,  1155,    12, 17648,    66,
           332, 17262,   343,     7,    12, 11419,     1,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,   

In [62]:
dump(tokenized_dataset_train_gflan, basepath+'train_set.joblib')
dump(tokenized_dataset_valid_gflan, basepath+'valid_set.joblib')
dump(tokenized_dataset_test_gflan, basepath+'test_set.joblib')

['/content/drive/MyDrive/Applied_NLP/test_set.joblib']

In [59]:
o = model_gflan(input_ids=tokenized_dataset_train_gflan['input_ids'][0].unsqueeze(0), attention_mask=tokenized_dataset_train_gflan['attention_mask'][0].unsqueeze(0), labels=tokenized_dataset_train_gflan[0]['labels'].unsqueeze(0))
o

Seq2SeqSequenceClassifierOutput(loss=tensor(0.6720, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>), logits=tensor([[-0.0355, -0.2100, -0.1591, -0.1174, -0.0737,  0.0747,  0.1554, -0.0178,
         -0.2763, -0.5346,  0.4524]], grad_fn=<AddmmBackward0>), past_key_values=None, decoder_hidden_states=None, decoder_attentions=None, cross_attentions=None, encoder_last_hidden_state=tensor([[[-0.0634,  0.1587, -0.1917,  ...,  0.1056,  0.1381,  0.0195],
         [ 0.0014,  0.0976, -0.2298,  ...,  0.0787, -0.0118,  0.0172],
         [-0.0236,  0.0376, -0.1628,  ...,  0.0963,  0.0437, -0.1908],
         ...,
         [-0.1518,  0.2759, -0.2498,  ...,  0.2594, -0.0370,  0.0139],
         [-0.1424,  0.2444, -0.2035,  ...,  0.3095, -0.0775,  0.0114],
         [-0.1424,  0.2444, -0.2035,  ...,  0.3095, -0.0775,  0.0114]]],
       grad_fn=<MulBackward0>), encoder_hidden_states=None, encoder_attentions=None)

In [ ]:
tr

In [37]:
trainset[0:5]

{'texts': ['@KMunie7 @KaranEsch Helluva lot more animated than they were for the actual game &gt;.&lt;',
  "Despite my sharing O'Neill's article, what saddens me about the Gazza case is there seems to be more fury about the sentence than the remark",
  '@mtailor “20% Better Than a Tailor”? Least a tailor would know #navy vs. #blue. $500 and 4 weeks for an #innacurate suit? ',
  'How on earth can the projection of all that is good and happy and true in my soul be a poisonous fucking snake? #patronus #adder #offended',
  'Patti seems so sad. She stamped and ran behind the sofa. We will have to give her plenty of love and affection...more than usual. '],
 'labels': [[0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0],
  [1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0],
  [1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0],
  [1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0]]}

In [38]:
testset[0:5]

{'texts': ['@Adnan__786__ @AsYouNotWish Dont worry Indian army is on its ways to dispatch all Terrorists to Hell',
  'Academy of Sciences, eschews the normally sober tone of scientific papers and calls the massive loss of wildlife a “biological annihilation',
  'I blew that opportunity -__- #mad',
  'This time in 2 weeks I will be 30... 😥',
  '#Deppression is real. Partners w/ #depressed people truly dont understand the depth in which they affect us. Add in #anxiety &amp;makes it worse'],
 'labels': [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]]}

In [39]:
trainset.features

{'texts': Value(dtype='string', id=None),
 'labels': Sequence(feature=Value(dtype='float64', id=None), length=-1, id=None)}

#Distil-bert as Checkpoint

In [40]:
checkpoint_google_flan = "google/flan-t5-base"
tokenizer_google_flan = AutoTokenizer.from_pretrained(checkpoint_google_flan)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


##Tokenize for distil-bert



In [41]:
def tokenize_fn_googleflan(batch):
  return tokenizer_google_flan(text = batch["texts"], truncation=True, padding="max_length",return_tensors = "pt", max_length = 128)

In [42]:
tokenized_dataset_train_gflan = trainset.map(tokenize_fn_googleflan,batched=True)
tokenized_dataset_valid_gflan = validset.map(tokenize_fn_googleflan,batched=True)
tokenized_dataset_test_gflan = testset.map(tokenize_fn_googleflan,batched=True)

Map:   0%|          | 0/5406 [00:00<?, ? examples/s]

Map:   0%|          | 0/2318 [00:00<?, ? examples/s]

Map:   0%|          | 0/3259 [00:00<?, ? examples/s]

In [43]:
tokenized_dataset_train_gflan

Dataset({
    features: ['texts', 'labels', 'input_ids', 'attention_mask'],
    num_rows: 5406
})

In [44]:
tokenized_dataset_train_gflan[0]

{'texts': '@KMunie7 @KaranEsch Helluva lot more animated than they were for the actual game &gt;.&lt;',
 'labels': [0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0],
 'input_ids': [3320,
  23815,
  202,
  23,
  15,
  940,
  3320,
  439,
  9,
  2002,
  427,
  860,
  11419,
  76,
  900,
  418,
  72,
  16822,
  145,
  79,
  130,
  21,
  8,
  1805,
  467,
  3,
  184,
  122,
  17,
  117,
  5,
  184,
  40,
  17,
  117,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1

In [47]:
tokenized_dataset_train_gflan = tokenized_dataset_train_gflan.remove_columns(["texts"])
tokenized_dataset_valid_gflan = tokenized_dataset_valid_gflan.remove_columns(["texts"])
tokenized_dataset_test_gflan = tokenized_dataset_test_gflan.remove_columns(["texts"])

ValueError: Column name ['texts'] not in the dataset. Current columns in the dataset: ['labels', 'input_ids', 'attention_mask']

In [49]:
tokenized_dataset_train_gflan

Dataset({
    features: ['labels', 'input_ids', 'attention_mask'],
    num_rows: 5406
})

In [51]:
tokenized_dataset_train_gflan.set_format(type='torch')
tokenized_dataset_valid_gflan.set_format(type='torch')
tokenized_dataset_test_gflan.set_format(type='torch')

In [52]:
tokenized_dataset_train_gflan[0]

{'labels': tensor([0., 1., 0., 0., 1., 1., 1., 0., 0., 0., 0.]),
 'input_ids': tensor([ 3320, 23815,   202,    23,    15,   940,  3320,   439,     9,  2002,
           427,   860, 11419,    76,   900,   418,    72, 16822,   145,    79,
           130,    21,     8,  1805,   467,     3,   184,   122,    17,   117,
             5,   184,    40,    17,   117,     1,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,   

In [53]:
tokenized_dataset_test_gflan

Dataset({
    features: ['labels', 'input_ids', 'attention_mask'],
    num_rows: 3259
})

##Model for distil-bert

In [45]:
print(checkpoint_google_flan)

google/flan-t5-base


In [57]:
id2labels

{0: 'anger',
 1: 'anticipation',
 2: 'disgust',
 3: 'fear',
 4: 'joy',
 5: 'love',
 6: 'optimism',
 7: 'pessimism',
 8: 'sadness',
 9: 'surprise',
 10: 'trust'}

In [58]:
model_gflan = AutoModelForSequenceClassification.from_pretrained(checkpoint_google_flan,num_labels = 11,problem_type="multi_label_classification",id2label=id2labels,label2id=labels2id)

Some weights of T5ForSequenceClassification were not initialized from the model checkpoint at google/flan-t5-base and are newly initialized: ['classification_head.dense.bias', 'classification_head.dense.weight', 'classification_head.out_proj.bias', 'classification_head.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [48]:
import evaluate
from transformers import EvalPrediction
import torch
from sklearn.metrics import f1_score,  accuracy_score

##Metrics labelling functions

In [49]:
def multi_label_metrics(predictions, labels, threshold=0.5):
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    accuracy = accuracy_score(y_true, y_pred)
    metrics = {'f1': f1_micro_average,'Accuracy': accuracy}
    return metrics

def compute_metrics_1(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions,tuple) else p.predictions
    result = multi_label_metrics(predictions=preds,labels=p.label_ids)
    return result

##Training Args

In [50]:
training_args = TrainingArguments(
    num_train_epochs=5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    learning_rate=0.00002,
    optim='adamw_torch',
    remove_unused_columns=False,
    output_dir=str(modelfolder),
    evaluation_strategy='steps',
    eval_steps=100,
    save_strategy="steps",
    save_steps=100,
    load_best_model_at_end=True,
    save_total_limit=2,
    metric_for_best_model="f1",
    greater_is_better=True,
    logging_strategy='steps',
    logging_steps=10,
    report_to='wandb',
    run_name='emoticon_hf_trainer',
)

##Bert Trainer

In [51]:
tokenized_dataset_train_gflan

Dataset({
    features: ['labels', 'input_ids', 'attention_mask'],
    num_rows: 5406
})

In [52]:
trainer_gflan = Trainer(
    model=model_gflan,
    args=training_args,
    train_dataset = tokenized_dataset_train_gflan,
    eval_dataset = tokenized_dataset_valid_gflan,
    compute_metrics=compute_metrics_1,
)


/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [53]:
!wandb login 5b0f2851a77f589c556f16598a0b181257a541e9

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [54]:
%env WANDB_PROJECT = nlp_course_spring_2024-emtoicons-analysis-hf-trainer

env: WANDB_PROJECT=nlp_course_spring_2024-emtoicons-analysis-hf-trainer


##Training

In [55]:
trainer_gflan.train()

wandb: Currently logged in as: abhishekmoguluri (abhishek-moguluri). Use `wandb login --relogin` to force relogin


Step,Training Loss,Validation Loss,F1,Accuracy
100,0.465500,0.475466,0.014100,0.022433


KeyboardInterrupt: 

In [ ]:
best_model_gflan_checkpoint_step = trainer_gflan.state.best_model_checkpoint.split('-')[-1]
print(f"The best model was saved at step {best_model_gflan_checkpoint_step}.")

The best model was saved at step 3300.


In [ ]:
wandb.finish()

eval/Accuracy,▁▁▂▅▆▆▆▆▇▇▇▇▇▇▇▇▇▇███████████████
eval/f1,▁▁▃▆▇▇▇▇▇▇▇██████████████████████
eval/loss,██▆▄▄▃▃▃▂▂▂▂▁▁▁▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eval/runtime,▆▄▂▃▁▂▂▁▃▄▄▁▂▂▃▂▂▁▂▂▂▄▃▂█▂▃▃█▂▄▅▂
eval/samples_per_second,▃▅▇▆█▇▇█▆▅▄█▇▇▆▇▆█▇▇▇▄▆▇▁▇▆▆▁▇▅▄▇
eval/steps_per_second,▃▅▇▆█▇▇█▆▅▄█▇▇▆▇▆█▇▇▇▄▆▇▁▇▆▆▁▇▅▄▇
train/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/grad_norm,▁▁▃▂▅▄▅▂▃▅▅▅▅▂▅▇▂▄▂▇█▄▅▆▃▃▅▃▄▃▂▂▅▃▄▆▅▇▄▄
train/learning_rate,███▇▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▁▁▁
train/loss,█▅▅▅▅▄▃▃▃▃▃▃▂▂▂▂▃▃▂▃▂▃▃▂▂▂▂▃▃▁▂▁▁▂▂▂▁▂▂▁


##Save the model

In [ ]:
model_gflan = str(modelfolder/f'checkpoint-{best_model_gflan_checkpoint_step}')
model_gflan

'/content/drive/MyDrive/Applied_NLP/Class_8/Home_Work_6/Model_Checkpoint/checkpoint-3300'

In [ ]:
model_gflan = AutoModelForSequenceClassification.from_pretrained(model_gflan)

##Validation set

In [ ]:
training_args_gflan_test = TrainingArguments(
    output_dir=str(modelfolder),
    per_device_eval_batch_size=8,
    do_train=False,
    do_eval=True,
    report_to='wandb',
    run_name='emoticon_hf_trainer',
)

In [ ]:
trainer_gflan_test = Trainer(
    model=model_gflan,
    args=training_args_gflan_test,
    eval_dataset=tokenized_dataset_test_gflan,
    compute_metrics=compute_metrics_1,)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [ ]:
gflan_test_results = trainer_gflan_test.evaluate()

In [ ]:
gflan_test_results

{'eval_loss': 0.44685861468315125,
 'eval_f1': 0.0,
 'eval_Accuracy': 0.05799324946302547,
 'eval_runtime': 18.8123,
 'eval_samples_per_second': 173.238,
 'eval_steps_per_second': 21.688}

In [ ]:
wandb.log({"test_accuracy": gflan_test_results["eval_Accuracy"], "test_loss": gflan_test_results["eval_loss"], "test_f1": gflan_test_results["eval_f1"]})

In [ ]:
gflan_test_output = trainer_gflan_test.predict(tokenized_dataset_test_gflan)
gflan_test_output

PredictionOutput(predictions=(array([[ 1.581794  , -3.128667  ,  1.5465628 , ..., -1.3538101 ,
        -3.8697073 , -4.9532194 ],
       [ 1.8910961 , -2.9422705 ,  2.1043222 , ..., -1.0229391 ,
        -3.2650368 , -4.5783806 ],
       [ 0.710735  , -4.0956335 ,  0.59855014, ...,  0.89036363,
        -4.0992684 , -5.2454834 ],
       ...,
       [ 3.268788  , -3.4757533 ,  2.8487258 , ..., -1.5727715 ,
        -3.6295354 , -5.1130695 ],
       [-4.1936584 , -2.625671  , -4.505632  , ..., -3.5758827 ,
        -3.2114654 , -1.7294221 ],
       [-2.8309588 , -1.330192  , -3.2103913 , ..., -3.4653013 ,
        -2.47794   , -2.4960203 ]], dtype=float32), array([[[-0.13719875,  0.17859243, -0.12330753, ...,  0.08664794,
          0.16714594,  0.00370717],
        [ 0.01131361,  0.18526857, -0.2551252 , ..., -0.03460981,
          0.1318605 , -0.14588004],
        [-0.07658564,  0.13461149, -0.10155066, ..., -0.06434876,
          0.05500609, -0.01644121],
        ...,
        [-0.26031083, 

In [ ]:
gflan_test_preds = gflan_test_output.predictions

In [ ]:
gflan_test_preds = torch.tensor(gflan_test_preds[0])

##Predictions to Probablities

In [ ]:
def coversion(predictions, threshold=0.25):
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    results = np.zeros(probs.shape)
    results[np.where(probs >= threshold)] = 1
    return results

In [ ]:
gflan_test_output_final = coversion(gflan_test_preds)
(gflan_test_output_final)

array([[1., 0., 1., ..., 0., 0., 0.],
       [1., 0., 1., ..., 1., 0., 0.],
       [1., 0., 1., ..., 1., 0., 0.],
       ...,
       [1., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
df_test = pd.read_csv(datafolder/"emoticons_test.csv")

In [ ]:
df_test.columns

Index(['ID', 'Tweet', 'anger', 'anticipation', 'disgust', 'fear', 'joy',
       'love', 'optimism', 'pessimism', 'sadness', 'surprise', 'trust'],
      dtype='object')

In [ ]:
df_test[['anger', 'anticipation', 'disgust', 'fear', 'joy', 'love',
       'optimism', 'pessimism', 'sadness', 'surprise', 'trust']] = gflan_test_output_final

In [ ]:
df_test.drop(["Tweet"],axis =1, inplace = True)

In [ ]:
df_test.to_csv(basefolder/"Class_8/Home_Work_6/Submissions/gdlan-test.csv")

##Inference

In [ ]:
mapped_labels =[]
for prediction_row in gflan_test_output_final:
    mapped_row = [label for label, pred in zip(labels, prediction_row) if pred]
    mapped_labels.append(mapped_row)

In [ ]:
from pprint import pprint
pprint(mapped_labels[:10])

[['anger', 'disgust', 'fear'],
 ['anger', 'disgust', 'sadness'],
 ['anger', 'disgust', 'fear', 'sadness'],
 ['anticipation', 'joy', 'optimism'],
 ['fear', 'pessimism', 'sadness'],
 ['anger', 'disgust', 'fear'],
 ['anticipation', 'joy', 'optimism'],
 ['joy', 'optimism'],
 ['joy', 'love', 'optimism'],
 ['fear', 'pessimism', 'sadness']]
